In [16]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('indicators'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import sqlite3
#from db_helper import *
import ta
from ttrpy.trend.sma import sma
from ttrpy.trend.dema import dema
from ttrpy.trend.ema import ema
from ttrpy.momentum.stoch import stoch
from ttrpy.momentum.mom import mom
from ttrpy.momentum.ppo import ppo

from indicators import SuperTrend
from finta import TA

import time

In [65]:
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

import warnings
warnings.simplefilter("ignore")

df = pd.read_csv('binance.csv')

df = df.rename(columns={
    'date': 'Date', 
    'open': 'Open', 
    'high': 'High', 
    'low': 'Low', 
    'close': 'Close', 
    'volume': 'Volume'
})

df['Timestamp'] = pd.to_datetime(df['Date']*1000000)
df = df[df['Date']>1572566400*1000]

df

,Date,Open,High,Low,Close,Volume,Timestamp
19439,1572570000000,9127.00,9141.70,9090.08,9092.65,992.634124,2019-11-01 01:00:00
19440,1572573600000,9092.71,9140.61,9062.09,9123.16,1379.955092,2019-11-01 02:00:00
19441,1572577200000,9122.85,9122.85,9043.85,9063.13,1559.194143,2019-11-01 03:00:00
19442,1572580800000,9064.12,9100.00,9030.00,9078.33,1155.043623,2019-11-01 04:00:00
19443,1572584400000,9077.63,9111.58,9072.28,9080.79,906.475974,2019-11-01 05:00:00
19444,1572588000000,9080.62,9137.00,9070.00,9118.90,1092.372598,2019-11-01 06:00:00
19445,1572591600000,9116.37,9127.50,9080.12,9099.00,1110.756136,2019-11-01 07:00:00
19446,1572595200000,9099.43,9174.99,9094.20,9134.63,2269.695176,2019-11-01 08:00:00
19447,1572598800000,9134.63,9136.00,9088.88,9103.12,1404.482801,2019-11-01 09:00:00
19448,1572602400000,9103.42,9181.32,9095.13,9155.93,2095.123101,2019-11-01 10:00:00


In [66]:
df = sma(df, 'Close', 'ma_32', 32)
df = sma(df, 'Close', 'ma_52', 52)
df = dema(df, 'Close', 'dema_13', 13)
df = dema(df, 'Close', 'dema_21', 21)
df = ema(df, 'Close', 'ema_13', 13)
df = ema(df, 'Close', 'ema_21', 21)

indicator_psar = ta.trend.PSARIndicator(high=df['High'], low=df['Low'], close=df['Close'], step=0.02, max_step=0.2)
df['psar'] = indicator_psar.psar()
df['psar_down'] = indicator_psar.psar_down()
df['psar_down_indicator'] = indicator_psar.psar_down_indicator()
df['psar_up'] = indicator_psar.psar_up()
df['psar_up_indicator'] = indicator_psar.psar_up_indicator()

df = SuperTrend(df=df, period=10, multiplier=3)

df = stoch(df=df, high='High', low='Low', close='Close', fast_k_n=8, slow_k_n=5, slow_d_n=3)

indicator_rsi = ta.momentum.RSIIndicator(close=df['Close'], n=8)
df['rsi'] = indicator_rsi.rsi()

indicator_uo = ta.momentum.UltimateOscillator(high=df['High'], low=df['Low'], close=df['Close'], s=8, m=13, len=21)
df['uo'] = indicator_uo.uo()

indicator_macd = ta.trend.MACD(close=df['Close'], n_fast=13, n_slow=21)
df['macd'] = indicator_macd.macd()
df['macd_diff'] = indicator_macd.macd_diff()
df['macd_signal'] = indicator_macd.macd_signal()

df = mom(df, 'Close', 'mom', 31)

indicator_adx = ta.trend.adx(high=df['High'], low=df['Low'], close=df['Close'], n=13)

df.rename(columns={'Close': 'close', 'High': 'high', 'Low': 'low'}, inplace=True)

df = pd.concat([df, TA.WTO(ohlc=df, channel_lenght=10, average_lenght=21)], axis=1)
df.rename(columns={'close': 'Close', 'high': 'High', 'low': 'Low'}, inplace=True)

df = ppo(df, 'Close', 'ppo', fast_period=10, slow_period=21, ma_type=0)

indicator_cci = ta.trend.CCIIndicator(high=df['High'], low=df['Low'], close=df['Close'], n=20, c=1)
df['cci'] = indicator_cci.cci()




df['Date'] = pd.to_datetime(df['Date'])
timestamp_index = pd.DatetimeIndex(df['Date'].values)
df = df.set_index(timestamp_index)


export_csv = df.to_csv(r'my_indicators.csv', index=None, header=True)